In [0]:
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from strategies import TotalSalesStrategy, TotalUnitsStrategy, MaxSalesStrategy
from charts_utils import generate_sales_bar_chart, generate_units_bar_chart

class SalesChatbot:
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.docs = [f"Product: {row['Product']}, Total Sales: {row['Total Sales']}, Units Sold: {row['Units Sold']}" 
                     for _, row in self.data.iterrows()]
        self.embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        self.vectorstore = FAISS.from_texts(self.docs, self.embedding_model)
        tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
        model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
        llm_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=-1)
        self.llm = HuggingFacePipeline(pipeline=llm_pipeline)
        self.strategies = {
            "total sales": TotalSalesStrategy(),
            "total units": TotalUnitsStrategy(),
            "highest sales": MaxSalesStrategy()
        }
        self.chat_history = []

    def answer_query(self, query):
        for key in self.strategies:
            if key in query.lower():
                answer = self.strategies[key].handle(self.data, query)
                self.chat_history.append({"user": query, "bot": answer})
                return answer
        if "chart" in query.lower() or "graph" in query.lower():
            sales_chart = generate_sales_bar_chart(self.data)
            units_chart = generate_units_bar_chart(self.data)
            return f"Generated charts:\n- {sales_chart}\n- {units_chart}"
        retrieved_docs = self.vectorstore.similarity_search(query, k=2)
        context = "\n".join([doc.page_content for doc in retrieved_docs])
        history_text = "\n".join([f"User: {turn['user']}\nBot: {turn['bot']}" for turn in self.chat_history])
        prompt = f"""
Answer the user query based on the sales report context.
Context:
{context}

Conversation History:
{history_text}

User: {query}
Bot:"""
        result = self.llm(prompt, max_length=200)[0]['generated_text'].strip()
        self.chat_history.append({"user": query, "bot": result})
        return result